In [1]:
import sys
import cv2
import numpy as np

#桶形畸变
def barrel_distortion(img, distortion):
    height, width = img.shape[:2]
    K1 = distortion
    cx = width / 2
    cy = height / 2

    map_x = np.zeros((height, width), np.float32)
    map_y = np.zeros((height, width), np.float32)

    for y in range(height):
        for x in range(width):
            dx = (x - cx) / cx
            dy = (y - cy) / cy
            r = np.sqrt(dx**2 + dy**2)
            factor = 1 + K1 * r**2

            map_x[y, x] = cx + dx * factor * cx
            map_y[y, x] = cy + dy * factor * cy

    distorted_img = cv2.remap(img, map_x, map_y, interpolation=cv2.INTER_LINEAR)
    return distorted_img

def draw_matches(left_img, kpleft, right_img, kpright, all_matches):
    left_row, left_col = left_img.shape[:2]
    right_row, right_col = right_img.shape[:2]

    match_img = np.zeros((max([left_row, right_row]), left_col + right_col, 3), dtype='uint8')
    match_img[:left_row, :left_col, :] = np.dstack([left_img, left_img, left_img])
    match_img[: right_row, left_col:left_col + right_col, :] = np.dstack([right_img, right_img, right_img])

    for m in all_matches:
        left_img_index = m.queryIdx
        right_img_index = m.trainIdx
        (xleft, yleft) = kpleft[left_img_index].pt
        (xright, yright) = kpright[right_img_index].pt

        cv2.circle(match_img, (int(xleft), int(yleft)), 4, (0, 255, 255), 1)
        cv2.circle(match_img, (int(xright) + left_col, int(yright)), 4, (0, 255, 255), 1)

        cv2.line(match_img, (int(xleft), int(yleft)), (int(xright) + left_col, int(yright)), (0, 255, 255), 1)

    return match_img


def crop_image(image):
    # 将黑色区域设置为mask
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

    # 查找外部轮廓
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 选择最大的轮廓
    max_area = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > max_area:
            max_area = area
            x, y, w, h = cv2.boundingRect(cnt)

    # 裁剪图像并返回
    cropped_image = image[y:y+h, x:x+w]
    return cropped_image

def match_images(left_img, right_img):
    orb = cv2.ORB_create(nfeatures=1000)
    kpleft, dcleft = orb.detectAndCompute(left_img, None)
    kpright, dcright = orb.detectAndCompute(right_img, None)

    brutef = cv2.BFMatcher_create(cv2.NORM_HAMMING)
    matches = brutef.knnMatch(dcleft, dcright, k=2)

    all_matches = []
    for m, n in matches:
        all_matches.append(m)

    good_matches = []
    threshold = 0.6
    for m, n in matches:
        if m.distance < threshold * n.distance:
            good_matches.append(m)

    if len(good_matches) > 15:
        left_kp_matchcontrol = [kpleft[match.queryIdx].pt for match in good_matches]
        right_kp_matchcontrol = [kpright[match.trainIdx].pt for match in good_matches]
        size = -1, 1, 2
        source = np.float32(left_kp_matchcontrol).reshape(size)
        destination = np.float32(right_kp_matchcontrol).reshape(size)

        homo, _ = cv2.findHomography(source, destination, cv2.RANSAC, 3.0)

        def match_images_internal(left_img, right_img, homography):
            left_row, left_col = left_img.shape[:2]
            right_row, right_col = right_img.shape[:2]
            default = 0

            points1 = np.float32([[default, default], [default, left_row], [left_col, left_row], [left_col, default]]).reshape(size)
            temp_points = np.float32([[default, default], [default, right_row], [right_col, right_row], [right_col, default]]).reshape(size)

            points2 = cv2.perspectiveTransform(temp_points, homography)

            points = np.concatenate((points1, points2), axis=0)

            [minx, miny] = np.int32(points.min(axis=0).ravel() - 0.5)
            [maxx, maxy] = np.int32(points.max(axis=0).ravel() + 0.5)

            distance = [-minx, -miny]

            homography_translation = np.array([[1, default, distance[default]], [default, 1, distance[1]], [default, default, 1]])

            result = cv2.warpPerspective(right_img, homography_translation.dot(homography), (maxx - minx, maxy - miny))
            result[distance[1]:left_row + distance[1], distance[default]:left_col + distance[default]] = left_img

            return result

        result = match_images_internal(right_img, left_img, homo)

        # 应用桶形畸变
        distortion_coefficient = 0.05 # 畸变程度
        result = barrel_distortion(result, distortion_coefficient)

        # 裁剪掉空白边缘
        result = crop_image(result)

        return result
    else:
        return None

def stitch_images(images):
    # 第一张图
    result_img = images[0]

    # 迭代图片列表，从第二张图开始拼接
    for i in range(1, len(images)):
        result_img = match_images(result_img, images[i])
        if result_img is None:
            print("Unable to match image {}.".format(i))
            return None

        # 裁切掉空白边缘
        result_img = crop_image(result_img)

    return result_img

# 读取图片列表
image_files = ["1.jpg", "2.jpg","4.jpg","5.jpg","6.jpg"]
images = [cv2.imread(img_file) for img_file in image_files]

# 拼接图片
stitched_img = stitch_images(images)

# 展示和保存结果
if stitched_img is not None:
    cv2.imwrite('final_result.jpg', stitched_img)
    cv2.imshow("final_result", stitched_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Unable to stitch the images together.")